In [10]:
import networkx as nx
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import random
import pickle
import osmnx as ox
import copy
import json
import os
import glob

In [11]:
"""
Input
"""
seed = 10
example = 8
num_nodes = 100
num_clients = 10
budget = 2
num_auto = 3
num_non_auto = 3
num_layer = 2

# Write the input to a file
input = ['seed', 'example', 'num_nodes', 'num_clients', 'budget', 'num_auto', 'num_non_auto', 'num_layer']
input_dict = dict(zip(input, [seed, example, num_nodes, num_clients, budget, num_auto, num_non_auto, num_layer]))
path = 'combined-mc-'+str(example)
if not os.path.exists(path):
    os.mkdir('combined-mc-' + str(example))
with open('combined-mc-' +str(example) + '/input.json', 'w') as outfile:
    json.dump(input_dict, outfile)



In [12]:
"""
Set the seed
"""
random.seed(seed)
np.random.seed(seed)

In [13]:
"""
Build a graph
"""

# Read the network data 
G = pickle.load(open("Manhattan_network.p", "rb"))


# Generate the set of nodes of a subgraph by BFS
def graph_generator(G, num_nodes):
    V = list(G.nodes)
    E = list(G.edges)
    s = random.sample(V, 1)[0]

    visited = [s]
    frontier = [s]
    while len(visited) < num_nodes and len(frontier) > 0:
        i = frontier.pop(0)
        for j in G.neighbors(i):
            if j not in visited:
                visited.append(j)
                frontier.append(j)
    
    return visited

tmp_V = graph_generator(G, num_nodes)
G = G.subgraph(tmp_V)


# Extract the strongly connected components
tmp_V = max(nx.strongly_connected_components(G), key = len)
G = G.subgraph(list(tmp_V))
V = list(G.nodes)
E = list(G.edges)

# label mapping
old_to_new = {}
new_to_old = {}
index = 0
for i in V:
    new_to_old[index] = i
    old_to_new[i] = index
    index += 1

In [14]:
"""
Set parameters (key control params, as input, are in the first chunk)
"""

## Depots
K = random.sample(V, 1)

## Trucks
num_truck = num_auto + num_non_auto ### The first half are auto trucks
truck = np.arange(num_truck)

truck_range = (np.random.random(len(truck))) * 50 * 1600
truck_capacity = 10

# Fixed cost to dispatch a truck
truck_cost = np.repeat(100, len(truck))


## Clients
D = random.sample(list(set(V) - set(K)), num_clients)

demand = np.random.lognormal(0.1, 1, size=len(D))
demand[demand >= truck_capacity] = truck_capacity


## Time discretization

# The length of time intervals (in minutes)
gran = 1
# The # of operational hours (in hours)
op = 3
# Time span (in seconds)
T = op * 3600

num_interval = int((60 / gran) * op)
Q = np.arange(num_interval)
a = np.arange(0, T, gran * 60)
b = a + gran * 60

## Extra cost for remote control
control_cost = np.repeat(10, num_auto)



In [15]:
"""
Build the extended graph
"""

# G_e = copy.deepcopy(G)
G_e = nx.convert_node_labels_to_integers(G)
D_e = [old_to_new[i] for i in D]
for i in range(num_layer - 1):

    # Duplicate one more layer
    # layer = copy.deepcopy(G)
    layer = nx.convert_node_labels_to_integers(G, first_label = (i+1) * len(V)) 
    G_e = nx.disjoint_union(G_e, layer)

    # Add artificial edges of customers 
    for j in D:
        idx_j = old_to_new[j]
        idx_1 = idx_j + i * len(V)
        idx_2 = idx_j + (i+1) * len(V)
        D_e.append(idx_2)
        G_e.add_edge(idx_1, idx_2, length = 0, highway = 'artificial', index = str(idx_1)+'-'+str(idx_2))

# Construct the demand dictionary
demand_dict = dict(zip(D_e, np.tile(demand, num_layer)))

# Construct the 2-D customer vertices
D_e_2d = np.reshape(D_e, (num_layer, len(D)))

# Add artificial edges of the depot
departure = old_to_new[K[0]]
K_e = []
for i in range(num_layer - 1):
    idx_1 = departure + i * len(V)
    idx_2 = departure + (num_layer - 1) * len(V)
    K_e.append(idx_1)
    G_e.add_edge(idx_1, idx_2, length = 0, highway = 'artificial', index = str(idx_1)+'-'+str(idx_2))
arrival = departure + (num_layer - 1) * len(V)
K_e.append(arrival)

# Make new lists of nodes and edges in the extended graph
E_e = list(G_e.edges())
V_e = list(G_e.nodes)


# Mapping for edges
edge_to_no = {}
no_to_edge = {}
index = 0
for e in E_e:
    no_to_edge[index] = e
    edge_to_no[e] = index
    index += 1

# Sanity Check 
counter = 0
for j in range(num_layer - 1):

    tmp_k = old_to_new[K[0]]
    print(G_e[tmp_k + j * len(V)][tmp_k + (num_layer-1) * len(V)])
    print('++++++++++++++++++++++')
    for i in D:
        tmp_i = old_to_new[i]
        print(G_e[tmp_i + j * len(V)][tmp_i + (j+1) * len(V)])
        counter += 1
    print('======================')

print("The size of the extended graph is: ")
print(len(E_e))
print(len(V_e))


{0: {'length': 0, 'highway': 'artificial', 'index': '54-144'}}
++++++++++++++++++++++
{0: {'length': 0, 'highway': 'artificial', 'index': '38-128'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '47-137'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '64-154'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '19-109'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '85-175'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '39-129'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '73-163'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '58-148'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '13-103'}}
{0: {'length': 0, 'highway': 'artificial', 'index': '67-157'}}
The size of the extended graph is: 
381
180


In [16]:
"""
Cost and travel time
"""


speed = 3.3528 # in meter/second
f1 = 0.8
f2 = 1


for e in E_e:
    e = (e[0], e[1], 0)
    G_e.edges[e]['travel_time'] = G_e.edges[e]['length'] / speed

for e in E_e:
    e = (e[0], e[1], 0)
    if G_e.edges[e]['highway'] == 'artificial':
        G_e.edges[e]['auto_cost'] = 0
        G_e.edges[e]['non_auto_cost'] = 0
    elif G_e.edges[e]['highway'] == 'primary' or G_e.edges[e]['highway'] == 'secondary':
        G_e.edges[e]['auto_cost'] = f1 * G_e.edges[e]['length']
        G_e.edges[e]['non_auto_cost'] = G_e.edges[e]['length']
    else:
        G_e.edges[e]['auto_cost'] = f2 * G_e.edges[e]['length']
        G_e.edges[e]['non_auto_cost'] = G_e.edges[e]['length']

In [17]:
"""
Compute all shortest paths among clients
"""
K_D = K + D
cost_non_auto = np.zeros((num_clients + 1, num_clients + 1))
# cost_auto = np.zeros((num_clients + 1, num_clients + 1))
path_non_auto = {}
# path_auto = {}

for i in range(len(K_D)):
    for j in range(len(K_D)):
        if i!=j:
            cost_non_auto[i][j] = nx.shortest_path_length(G, K_D[i], K_D[j], weight='non_auto_cost')
            # cost_auto[i][j] = nx.shortest_path_length(G, K_D[i], K_D[j], weight='auto_cost')

            path_non_auto[str(K_D[i])+'-'+str(K_D[j])] = nx.shortest_path(G, K_D[i], K_D[j], weight='non_auto_cost')
            # path_auto[str(K_D[i])+'-'+str(K_D[j])] = nx.shortest_path(G, K_D[i], K_D[j], weight='auto_cost')
            

In [18]:
"""
Solver (metric closure + exact model)
"""

model = gp.Model('combined_mc')

# Set the file path for logging
path = 'combined-mc-' + str(example) + '/combined_mc_log.txt'
if os.path.exists(path):
    os.remove(path)
model.Params.LogFile = path

# Parameters tuning for the MIP solver
model.Params.MIPFocus = 3 # Focusing on the phase 2 (getting good quality solutions)
# model.Params.MIPGap = 0.05 # Specify the terminating duality gap


# Create variables for auto trucks
x_e = model.addMVar((len(E_e), num_auto), vtype=GRB.BINARY)
y_e = model.addMVar((len(V_e), num_auto), vtype=GRB.BINARY)
t = model.addMVar((len(V_e), num_auto), vtype=GRB.CONTINUOUS)
u = model.addMVar((num_interval, num_auto), vtype=GRB.BINARY)
alpha = model.addMVar((num_interval, len(E_e), num_auto), vtype=GRB.BINARY)
beta = model.addMVar((num_interval, len(E_e), num_auto), vtype=GRB.BINARY)

# Create variables for non-auto trucks
x_mc = model.addMVar((len(K_D), len(K_D), num_non_auto), vtype=GRB.BINARY)
y_mc = model.addMVar((len(K_D), num_non_auto), vtype=GRB.BINARY)
h = model.addMVar((len(D), num_non_auto), vtype=GRB.CONTINUOUS, ub=sum(demand))



# Constraints for auto trucks

## Flow conservation at all nodes (except the depot)
for i in list(set(V_e) - set([departure, arrival])):
    out_no = [edge_to_no[e] for e in list(G_e.out_edges(i))]
    in_no = [edge_to_no[e] for e in list(G_e.in_edges(i))]
    model.addConstrs(sum(x_e[out_no, m]) == sum(x_e[in_no, m]) for m in truck[: num_auto])
    model.addConstrs(sum(x_e[out_no, m]) <= 1 for m in truck[: num_auto])


## Flow conservation at the depot
departure_out = [edge_to_no[e] for e in list(G_e.out_edges(departure))]
arrival_in = [edge_to_no[e] for e in list(G_e.in_edges(arrival))]

model.addConstrs(sum(x_e[departure_out, m]) == y_e[departure, m]
                for m in truck[: num_auto])
model.addConstrs(sum(x_e[arrival_in, m]) == y_e[arrival, m]
                for m in truck[: num_auto])
model.addConstrs(y_e[departure, m] == y_e[arrival, m]
                for m in truck[: num_auto])

## The # of truck used
model.addConstr(sum(y_e[departure, :]) <= num_auto)


## Allow a vehicle to pass thru a customer without serving it 
for i in D_e:
    i_in = [edge_to_no[e] for e in list(G_e.in_edges(i))]
    model.addConstrs(sum(x_e[i_in, m]) >= y_e[i, m] for m in truck[: num_auto])

#* This constraint is excluded since it is coupled with the constraints for non-auto trucks
### Each truck serves only one customer
# model.addConstrs(sum(y[i, m] for i in D_e_2d[:, h] for m in truck) == 1 for h in range(len(D)))

## Transition
for i in D_e:
    for j in D_e:
        if G_e.has_edge(i, j) and G_e[i][j][0]['highway'] == 'artificial':
            model.addConstrs(x_e[edge_to_no[(i, j)], m] == y_e[i, m] for m in truck[: num_auto])

## Range limit
model.addConstrs(sum(G_e[i][j][0]['length'] * x_e[edge_to_no[(i, j)], m] for i in V_e for j in V_e if G_e.has_edge(i, j)) <= truck_range[m]
                for m in truck[: num_auto])


## Capacity limit
model.addConstrs(sum(demand_dict[i] * y_e[i, m] for i in D_e) <= truck_capacity
                for m in truck[: num_auto])


## Subtour elimination + time tracking
for i in V_e:
    for j in V_e:
        if G_e.has_edge(i, j):
            model.addConstrs(t[j, m] >= t[i, m] + G_e[i][j][0]['travel_time'] + T * (x_e[edge_to_no[(i, j)], m] - 1) for m in truck[: num_auto])
model.addConstrs(t[arrival, m] == t[departure, m] + sum(G_e[i][j][0]['travel_time'] * x_e[edge_to_no[(i, j)], m] for i in V_e for j in V_e if G_e.has_edge(i, j))
                for m in truck[: num_auto])
model.addConstrs(t[departure, m] == 0 for m in truck[: num_auto])


## Budget of remote control
model.addConstrs(sum(u[q, :]) <= budget for q in Q)

## Time consistency of a truck being remotely controlled
for i in V_e:
    for j in V_e:
        if G_e.has_edge(i, j) and (G_e[i][j][0]['highway'] != 'primary' and G_e[i][j][0]['highway'] != 'secondary'):
            model.addConstrs(3 * u[q, m] >= alpha[q, edge_to_no[(i, j)], m] + beta[q, edge_to_no[(i, j)], m] + x_e[edge_to_no[(i, j)], m] - 2
                            for q in Q
                            for m in truck[: num_auto])
            model.addConstrs(t[j, m] - T * (alpha[q, edge_to_no[(i, j)], m]) <= a[q]
                            for q in Q
                            for m in truck[: num_auto])
            model.addConstrs(t[i, m] >= b[q] - T * (beta[q, edge_to_no[(i, j)], m])
                            for q in Q
                            for m in truck[: num_auto])

# # Constraints for non-auto trucks

# for i in range(len(K_D)):
#     model.addConstrs(sum(x_mc[:, i, m - num_auto]) == sum(x_mc[i, :, m - num_auto]) for m in truck[num_auto :])
#     model.addConstrs(x_mc[i, i, m - num_auto] == 0 for m in truck[num_auto :])
#     model.addConstrs(sum(x_mc[:, i, m - num_auto]) == y_mc[i, m - num_auto] for m in truck[num_auto :])

# #* This constraint is commented since it is coupled with the constraints for auto trucks
# # model.addConstrs(sum(y[i, :]) == 1 for i in range(1, len(K_D)))

# model.addConstr(sum(y_mc[0, :]) <= num_non_auto)

# model.addConstrs(sum(x_mc[i, j, m - num_auto] * cost_non_auto[i][j] for i in range(len(K_D)) for j in range(len(K_D))) <= truck_range[m] 
#                    for m in truck[num_auto :])

# model.addConstrs(sum(demand[i] * y_mc[i+1, m - num_auto] for i in range(len(D))) <= truck_capacity
#                 for m in truck[num_auto :])

# for i in range(len(D)):
#     for j in range(len(D)):
#         model.addConstrs(h[j, m - num_auto] >= h[i, m - num_auto] + sum(demand) * (x_mc[i+1, j+1, m - num_auto] - 1) + demand[i] for m in truck[num_auto :])
# model.addConstrs(h[i, m - num_auto] >= demand[i] for m in truck[num_auto :] for i in range(len(D)))


# The coupling constraint (each customer can only be served by one truck)
model.addConstrs(sum(y_e[i, m] for i in D_e_2d[:, l] for m in truck[: num_auto]) + sum(y_mc[l+1, :]) == 1
                for l in range(len(D)))

# The objective
model.setObjective(sum(x_e[edge_to_no[(i, j)], m] * G_e[i][j][0]['auto_cost'] for i in V_e for j in V_e for m in truck[: num_auto] if G_e.has_edge(i, j)) + 
                sum(x_mc[i, j, m - num_auto] * cost_non_auto[i][j] for i in range(len(K_D)) for j in range(len(K_D)) for m in truck[num_auto :]) + 
                y_e[departure, :] @ truck_cost[: num_auto] +
                y_mc[0, :] @ truck_cost[num_auto :] +
                   
                #* The last term by itself accounts for the extra cost of remote control
                #* The last term can also avoid unneccesary positive u_{qm}
                #* Adding this term affect the computational efficiency
                sum(u[q, :] @ control_cost for q in Q),

                GRB.MINIMIZE)

# Solve the VRP
model.optimize()

#! Work-in process
#! There are bugs in this solver 



Set parameter LogFile to value "combined-mc-8/combined_mc_log.txt"
Set parameter MIPFocus to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 195293 rows, 414669 columns and 524103 nonzeros
Model fingerprint: 0x5c22bfc3
Variable types: 570 continuous, 414099 integer (414099 binary)
Coefficient statistics:
  Matrix range     [4e-01, 1e+04]
  Objective range  [1e+00, 8e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 6e+04]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.03 work units)
Thread count was 1 (of 32 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
